In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()


In [2]:
import time
import sys
import cv2
import pygame as pg
import numpy as np

import motion_control
import vision
import kalman
import AlgoGlobNav as gn

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
vid = cv2.VideoCapture(1)


In [4]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500):
    """
    Sets the motor speeds of the Thymio
    param l_speed: left motor speed
    param r_speed: right motor speed
    """
    global motor_left_target, motor_right_target

    motor_left_target = l_speed
    motor_right_target = r_speed

In [5]:
@tdmclient.notebook.sync_var
def read_motors():
    """
    Read the motor speeds of the Thymio
    """
    global motor_left_speed, motor_right_speed

    return [motor_left_speed, motor_right_speed]


In [6]:
@tdmclient.notebook.sync_var
def read_proximity():
    """
    Read the proximity sensors of the Thymio
    """
    global prox_horizontal

    return prox_horizontal

In [7]:
%%run_python
nf_leds_prox_h(0,0,0,0,0,0,0,0)
nf_leds_prox_v(0,0)
nf_leds_rc(0)
nf_leds_sound(0)
nf_leds_temperature(0,0)
nf_leds_top(0,0,0)
nf_leds_bottom_left(0,0,0)
nf_leds_bottom_right(0,0,0)
nf_leds_buttons(30,0,0,0)
nf_leds_top(0,255,0)


In [8]:

# Kalman filter
t = time.time()
print("Start Kalman filter initialization")
mus_pos = []
covs_pos = []
mus_att = []
covs_att = []

meas_variance_pos = np.array([[0.001, 0],
                              [0, 0.001]])
meas_variance_vel = np.array([[0.1, 0],
                              [0, 0.1]])
meas_variance_att = np.array([0.001])
meas_variance_omega = np.array([0.1])

# Initialize state estimate using vision
pose, orientation = kalman.init_state(vid)

# Initialize kalman filters
try:
    kf_pos = kalman.Kalman(_acc_variance=0.5, _type="pose", init_x=pose)
    kf_att = kalman.Kalman(_acc_variance=0.5, _type="orientation", init_x=orientation)
except:
    print("Could not initialize Kalman filters")

f"Initialization complete: {time.time()-t}s"
print("Initial pose:", kf_pos.mean)
print("Initial orientation", kf_att.mean*180/np.pi)
print("--------------------------------------")


# Global navigation
t = time.time()
print("Start global navigation initialization")

optimal_pathP = gn.opt_path(vid)
optimal_path = np.array(optimal_pathP)*vision.fieldWidthM/vision.fieldWidthP

f"Initialization complete: {time.time()-t}s"
print("--------------------------------------")


# Motion control
t = time.time()
print("Start motion control initialization")

mc = motion_control.MotionControl(optimal_path*100,
                                  kf_pos.mean[0]*100,
                                  kf_pos.mean[1]*100,
                                  kf_att.mean[0])

f"Initialization complete: {time.time()-t}s"
print("--------------------------------------")


# Initialize pygame
pg.init()

h = 841 / 2
w = 1189 / 2
scr = pg.display.set_mode((w, h))

thymio = pg.image.load("thymio.png")
thymio = pg.transform.scale(thymio, (50, 50))

# main loop
running = True
simulate = False
local = False
while running:
    # Configure key presses for pygame and cv2 windows
    for event in pg.event.get():
        if event.type == pg.QUIT:
            cv2.destroyAllWindows()
            pg.quit()
            sys.exit()
        elif event.type == pg.KEYDOWN:
            if event.key == pg.K_RETURN :
                simulate = not simulate
    # Set up environment
    scr.fill((0, 0, 0))
    pg.draw.line(scr, (255, 255, 255), (0, int(h/2)), (w, int(h/2)))
    pg.draw.line(scr, (255, 255, 255), (int(w/2), 0), (int(w/2), h))

    # Start thymio movement
    if simulate:
        # Save expected values and covariance matrices
        mus_pos.append(kf_pos.mean)
        covs_pos.append(kf_pos.cov)
        mus_att.append(kf_att.mean)
        covs_att.append(kf_att.cov)

        # Read sensor data
        fetched = False
        i = 0
        while (not fetched) and (i < 10):
            video_odometry = vision.fetchOdoMeters(vid)
            if type(video_odometry) != bool:
                pose = video_odometry[0:2]
                orientation = np.array([video_odometry[2]])
                fetched = True
            i += 1
        if not fetched:
            print("No vision measurements found")

        motor_speeds = read_motors()
        speed = (sum(motor_speeds)/2)*0.0004                                # magnitude of velocity [m/s]
        angular_speed = (motor_speeds[1]-motor_speeds[0])*0.0004/0.094      # magnitude of angular velocity [rad/s]
        velocity = [speed*np.cos(mus_att[-1][0]), speed*np.sin(mus_att[-1][0])]

        # Estimate pose
        kf_pos.predict(dt=time.time()-t)                    # Prediction step
        kf_pos.update(meas_value=velocity,                  # Update step using linear velocity measurement
               meas_variance=meas_variance_vel,
               C=np.array([[0, 0, 1, 0], [0, 0, 0, 1]]))
        if fetched:
            kf_pos.update(meas_value=list(pose),             # Update step using position measurement
                meas_variance=meas_variance_pos,
                C=np.array([[1, 0, 0, 0], [0, 1, 0, 0]]))

        # Estimate orientation
        kf_att.predict(dt=time.time()-t)                    # Prediction step
        kf_att.update(meas_value=angular_speed,             # Update step using angular velocity measurement
               meas_variance=meas_variance_omega,
               C=np.array([0, 1]))
        if fetched:
            kf_att.update(meas_value=orientation[0],        # Update step using angle measurement
                meas_variance=meas_variance_att,
                C=np.array([1, 0]))

        # Motion control
        prox_front = read_proximity()
        if any(prox_front):
            local = True
            mc.update_local(kf_pos.mean[0]*100, kf_pos.mean[1]*100, kf_att.mean[0], prox_front)
        else:
            if local:
                optimal_pathP = gn.opt_path(vid)
                optimal_path = np.array(optimal_pathP)*vision.fieldWidthM/vision.fieldWidthP
                mc = motion_control.MotionControl(optimal_path*100,
                                  kf_pos.mean[0]*100,
                                  kf_pos.mean[1]*100,
                                  kf_att.mean[0])
            local = False
            mc.update_global(kf_pos.mean[0]*100, kf_pos.mean[1]*100, kf_att.mean[0])

        # Update motor speeds
        motors(l_speed=int(mc.l_speed), r_speed=int(mc.r_speed))

        # Draw Thymio and optimal path in window
        kalman.move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        kalman.drawLine([mu[0] for mu in mus_pos], [mu[1] for mu in mus_pos], w, h, scr)
        kalman.path(optimal_path, w, h, scr)
    else:
        kalman.move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        kalman.drawLine([mu[0] for mu in mus_pos], [mu[1] for mu in mus_pos], w, h, scr)
        motors(0, 0)

    # Update windows
    while(True):
        terrain = vision.terrainFetch(vid)
        if type(terrain) != bool:
            nodes, nodeCon, maskObsDilated = terrain
            break
    output = vision.liveFeedback(vid, nodes, nodeCon, maskObsDilated, optimal_pathP)
    if type(output) != bool:
        cv2.imshow('Live Feedback', output)

    pg.display.flip()
    t = time.time()

motors(0, 0)
vid.release()

Initial pose: [0.91984375 0.47831875 0.         0.        ]
Initial orientation [176.92479482   0.        ]
---------------------
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can not find goal position
Can no

SystemExit: 

c:\users\timme\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
motors(100, 100)
stop()

NameError: name 'motors' is not defined

In [4]:

motors(0,0)

NameError: name 'motors' is not defined

In [2]:
stop()